In [1]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver # import the memory to store the context

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]


@tool
def search_tool(query: str):
    """search the web for info using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun()
    result = search_tool.invoke(query)
    return result

tools = [search_tool]
tool_node = ToolNode(tools)

# setup llm node using ollama3.2
model = ChatOllama(model="llama3.2:latest").bind_tools(tools)

def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

In [2]:
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [3]:
# build the graph with memory
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# add edge from start to agent
workflow.add_edge(START, "agent")

# add conditional edge from agent to tools or end
workflow.add_conditional_edges("agent", should_continue, )

# add edge from tools to agent
workflow.add_edge("tools", "agent")

# initialize a memory
memory = MemorySaver()

# compile the graph
app = workflow.compile(checkpointer=memory)

In [ ]:
# use a unique thread_id for user
config = {"configurable": {"thread_id": "session_1"}}

response = app.invoke({
    "messages": [{
        "role": "user",
        "content": "Hello,  i am aashik thakur."}]
    },
    config=config)

print(response["messages"][-1].content)

Hello Aashik, I couldn't find any specific information about an individual named Aashik Thakur. It's possible that there may be multiple people with this name, and without more context or details, it's difficult to provide a precise answer. If you could provide more information about yourself, such as your profession or interests, I'd be happy to try and help further!


In [ ]:
# using the same thread_id to get the context
config = {"configurable" : {"thread_id": "session_1"}}

response = app.invoke({"messages": [{"role": "user", "content": "what is my name?"}]}, config=config)

print(response["messages"][-1].content)


Your name is Aashik Thakur.


In [ ]:
# using the same thread_id to get the context
config = {"configurable" : {"thread_id": "session_1"}}

response = app.invoke({"messages": [{"role": "user", "content": "how is the weather right nnow in kathmandu?"}]}, config=config)

print(response["messages"][-1].content)


The current weather in Kathmandu is Mist with a real-time temperature of 18°C, humidity of 26%, wind speed of 6.5km/h, and visibility of 4.5km.


In [13]:
current_state = app.get_state(config)

print(f"Values in state: {len(current_state.values['messages'])} message found.")

#list all the messages
for i, msg in enumerate(current_state.values['messages']):
    print(f"Message {i}: {msg.content}")

Values in state: 19 message found.
Message 0: What is the capital of France?
Message 1: 
Message 2: 2 weeks ago - Paris is the capital and largest city of France , with an estimated city population of 2,048,472 in an area of 105.4 km2 (40.7 sq mi), and a metropolitan population of 13,171,056 as of January 2025. Located on the river Seine in the centre of the Île-de-France region, it is the largest metropolitan ... June 23, 2025 - This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944. ... Paris (987–1419), the residence of the Kings of France, although they were consecrated at Reims. Orléans (1108), one of the few consecrations of a French monarch to ... 3 days ago - Its capital, largest city and main cultural and economic centre is Paris. Metropolitan France was settled during the Iron Age by Celtic tribes known as Gauls before Rome annexed the area in 51 BC, leading to a distinct Gallo-Roman culture. September 16, 2025 - 